<a href="https://colab.research.google.com/github/basolu/Basolu-Machine-Learning/blob/main/user_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.models import model_from_json
import numpy as np
import os
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from numpy import save
from math import*
from sklearn.metrics import classification_report
from keras.models import Model
from keras import regularizers
import matplotlib.pyplot as plt
from tensorflow.keras import layers

action = int(input("Run training (yes-1 \ no-0)?"))

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + group + '/Inertial Signals/'
  # load all 9 files as a single array
  filenames = list()
  # total acceleration
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
  # body acceleration
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  # body gyroscope
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file(prefix + group + '/y_'+group+'.txt')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
  # load all train
  trainX, trainy = load_dataset_group('train', prefix + 'drive/MyDrive/HARDataset/')
  #print("Shape trainX and trainy:", trainX.shape, trainy.shape)
  # load all test
  testX, testy = load_dataset_group('test', prefix + 'drive/MyDrive/HARDataset/')
  subjects = read_csv('drive/MyDrive/HARDataset/train/subject_train.txt', header=None, delim_whitespace=True)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  #print("Shape train and test:",trainX.shape, trainy.shape, testX.shape, testy.shape)
  #print(trainX.shape[1], trainX.shape[2], trainy.shape[1]) 
  return trainX, trainy, testX, testy

def autoencoder_model(X):
  trainX = X
  model = Sequential(
    [
      layers.Input(shape=(trainX.shape[1], trainX.shape[2])),
      layers.Conv1D(
          filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
      layers.Dropout(rate=0.2),
      layers.Conv1D(
          filters=16, kernel_size=7, padding="same", strides=2, activation="relu"),
      layers.Conv1DTranspose(
          filters=16, kernel_size=7, padding="same", strides=2, activation="relu"),
      layers.Dropout(rate=0.2),
      layers.Conv1DTranspose(
          filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
      layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)
  return model

#training-set for user 1's walking
def fun_target_train():
  trainX, trainy, testX, testy = load_dataset()
  subject = load_file("drive/MyDrive/HARDataset/train/subject_train.txt")
  #training only user n°2's data
  train_data = np.ndarray([])
  #train_target = np.ndarray([])
  size = 0
  for i in range(0, len(subject)):
      if (subject[i] == 2 and trainy[i][0]):
        size +=1
  train_data = np.empty([size, trainX.shape[1], trainX.shape[2]])
  #train_target = np.empty([size, trainy.shape[1]])
  count = 0
  for i in range(0, len(subject)):
      if (subject[i] == 2 and trainy[i][0]): 
        train_data[count] = trainX[i]
        #train_target[count] = trainy[count]
        count += 1 
  print("Number of train data:", size)
  np.save('drive/MyDrive/HARDataset/train_data.npy', train_data)
  #np.save('drive/MyDrive/HARDataset/train_target.npy', train_target)
  return train_data#train_target

#testing-set for user 2's walking
def fun_target_test():
  trainX, trainy, testX, testy = load_dataset()
  subject = load_file("drive/MyDrive/HARDataset/test/subject_test.txt")
  print(subject)
  #training only user n°2's data
  test_data = np.ndarray([])
  size = 0
  for i in range(0, len(subject)):
      if (subject[i] == 2 and testy[i][0]):
        size +=1
  test_data = np.empty([size, testX.shape[1], testX.shape[2]])
  count = 0
  for i in range(0, len(subject)):
      if (subject[i] == 2 and testy[i][0]): 
        test_data[count] = testX[i]
        count += 1
  print("Number of test data:", size)
  np.save('drive/MyDrive/HARDataset/test_data.npy', test_data)
  return test_data

def run_training():
  trainX, trainy, testX, testy = load_dataset()
  train_data = fun_target_train()
  test_data = fun_target_test()
  model = autoencoder_model(train_data)
  model.compile(optimizer='adam', loss='mae')
  #model.summary()
  nb_epochs = 50
  batch_size = 10
  history = model.fit(train_data, train_data, epochs=nb_epochs, batch_size=batch_size, validation_split=0.05).history
  print("Training completed")
  #save the trained model
  model_json = model.to_json()
  with open("drive/MyDrive/HARDataset/model_target.json", "w") as json_file:
    json_file.write(model_json)
  #serialize weights to HDF5
  model.save_weights("drive/MyDrive/HARDataset/model_target.h5")
  print("Saved model to disk")

def detect_anomalies():
  trainX, trainy, testX, testy = load_dataset()
  train_data = np.load('drive/MyDrive/HARDataset/train_data.npy')
  test_data  = np.load('drive/MyDrive/HARDataset/test_data.npy')
  print("Dataset loaded")
  json_file = open("drive/MyDrive/HARDataset/model_HV.json", "r")
  loaded_model_json = json_file.read()
  json_file.close()
  #load model from a json model
  model = model_from_json(loaded_model_json)
  # load weights into new model
  model.load_weights("drive/MyDrive/HARDataset/model_HV.h5")
  print("Loaded model from disk")  
  
  train_data_pred = model.predict(train_data)
  train_mae_loss = np.mean(np.abs(train_data_pred - train_data), axis=1)
  train_mae_loss = train_mae_loss.reshape((-1))
  plt.hist(train_mae_loss, bins=50)
  plt.xlabel("test MAE loss")
  plt.ylabel("No of samples")
  plt.show()
  # Detect all the samples which are anomalies.
  threshold = np.max(train_mae_loss)
  anomalies = train_mae_loss > threshold
  print("Reconstruction error threshold: ", threshold)
  print("Number of anomaly samples: ", np.sum(anomalies))
  print("Indices of anomaly samples: ", np.where(anomalies))

  test_data_pred = model.predict(test_data)
  test_mae_loss = np.mean(np.abs(test_data_pred - test_data), axis=1)
  test_mae_loss = test_mae_loss.reshape((-1))
  threshold_test = np.max(test_mae_loss)

  plt.hist(test_mae_loss, bins=50)
  plt.xlabel("test MAE loss")
  plt.ylabel("No of samples")
  plt.show()

  # Detect all the samples which are anomalies.
  anomalies = test_mae_loss > threshold_test
  print("Number of anomaly samples: ", np.sum(anomalies))
  print("Indices of anomaly samples: ", np.where(anomalies))

  # Checking how the first sequence is learnt

  plt.plot(trainX[0])
  plt.plot(trainX_pred[0])
  plt.show()

if (action == 1):
  run_training()
detect_anomalies()


Run training (yes-1 \ no-0)?1
Number of train data: 0
[[ 2]
 [ 2]
 [ 2]
 ...
 [24]
 [24]
 [24]]
Number of test data: 59


ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
